In [2]:
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Conv3D, Conv2DTranspose, Conv3DTranspose, Flatten, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt
import os
import Hyperparameters as hyp

In [3]:
test_path = 'Data/Testing/'
models = ["Base_Auto", "3DConv_Auto", 'LSTMAutoEncoderModel']

In [4]:
def getData(path, dim):
    data = []
    for file in os.listdir(path):
        print(file)
        if file[-3:] == "npy":
            frames = np.load(path + file, allow_pickle=True)
            clip = []
            for frame in frames: 
                clip.append(frame)
                if len(clip) == hyp.CLIP_LEN:
                    data.append(np.array(clip))
                    clip = []
    return np.array(data)

In [5]:
def computePreds(rec, x, y):
    C = np.array([np.linalg.norm(rec[i] - x[i]) for i in range(rec.shape[0])])
    Z = (C - np.mean(C)) / np.std(C)
    probs = np.ones(Z.shape[0]) - (1/(1 + np.exp(-Z)))
    n = int(probs.shape[0] * hyp.HIGHLIGHT_RATIO)
    pos_ind = set(probs.argsort()[-n:][::-1])
    y_hat = np.array([1 if i in pos_ind else 0 for i in range(probs.shape[0])])
#     y_hat = np.array([1 if probs[i] > 0.75 else 0 for i in range(probs.shape[0])])
    return y_hat

In [6]:
def plot(y_hat, y):
    plt.figure(figsize=(8, 6))
    plt.ylabel('Classification')
    plt.xlabel('Clip number')
    plt.plot(y_hat, label='Predictions')
    plt.plot(y, label="Actual")
    plt.legend()
    plt.show()

In [7]:
# def mse(y_hat, y):
#     return np.sum((y_hat - y)**2) / y.shape[0]

In [8]:
def get_eval_metrics(y_hat, y):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(y.shape[0]):
        if y[i] == 1 and y_hat[i] == 1:
            TP += 1
        elif y[i] == 1 and y_hat[i] == 0:
            FN += 1
        elif  y[i] == 0 and y_hat[i] == 1:
            FP += 1
        else:
            TN += 1
    A = float(TP + TN) / float(TP + TN + FN + FP)
    A_0 = float(TN) / float(TN + FP)
    A_1 = float(TP) / float(TP + FN)
    A_bar = 0.5 * (A_0 + A_1)
    
    A = float(TP + TN)/float(TP + TN + FP + FN)
    
    R = float(TP)/float(TP + FN)
    return A, A_bar, R

In [ ]:
test_data = getData(test_path, (32, 32))
test_flat = np.array([np.ravel(test_data[i]) for i in range(test_data.shape[0])])
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], test_data.shape[2], test_data.shape[3], 1))
print(f"test_flat shape is {test_flat.shape}")
print(f"test_data shape is {test_data.shape}")
test_y1 = np.genfromtxt(test_path + 'WolvesArsenal042419.csv', delimiter=",")
test_y2 = np.genfromtxt(test_path + 'BayRmd072119.csv', delimiter=",")
test_y2 = np.delete(test_y2, 1, 1)
test_y = np.append(test_y1, test_y2)
temp = []
for j in range(test_y.shape[0]):
    for k in range(3):
        temp.append(test_y[j])
while len(temp) > test_data.shape[0]:
    temp.pop()
test_y = np.array(temp)
print(f"Test_y shape is {test_y.shape}")
for i in range(len(models)):
    model = load_model(models[i])
    if i == 0:
        rec = model.predict(test_flat)
        preds = computePreds(rec, test_flat, test_y)
    else:
        rec = model.predict(test_data)
        preds = computePreds(rec, test_data, test_y)
    preds = preds.reshape((preds.shape[0], 1))
    plot(preds, test_y)
    print(f"The Loss for Model: {models[i]} is {loss}.")
    A, A_bar, recall = get_eval_metrics(preds, test_y)
    print(f"The accuracy(A) for Model: {models[i]} is {A}.")
    print(f"The weighted accuracy(A_Bar) for Model: {models[i]} is {A_bar}.")
    print(f"The recall(R) for Model: {models[i]} is {recall}.")

WolvesArsenal042419.csv
WolvesArsenal042419.npy
.ipynb_checkpoints
BayRmd072119.npy
BayRmd072119.csv
(1256, 10, 32, 32)
test_flat shape is (1256, 10240)
test_data shape is (1256, 10, 32, 32, 1)
Test_y shape is (420,)
Test_y shape is (1256,)
